In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
import pandas as pd
import plotly.graph_objects as go
import utm

from filtering import Filtering

# set the pandas plotting backend to plotly
pd.options.plotting.backend = "plotly"


## Read in the Data


In [3]:
df = pl.scan_parquet(
    # "/Users/max/Library/CloudStorage/Box-Box/Radar-Data/1677797903256.parquet"
    # "/Users/max/Downloads/1677797903256.parquet"
    "/Users/max/Library/CloudStorage/Box-Box/Radar-Data/new_format/1678658102006.parquet"
)


In [4]:
df = df.collect()


In [5]:
# add the object id & sort by time
df = df.with_columns(
    [
        (pl.col("ui32_objectID").cast(str) + "_" + pl.col("ip")).alias("object_id"),
    ]
).sort("epoch_time")


### Resample the Radar Data to Every .15 Seconds


In [6]:
GROUPBY_EVERY = "100ms"

df = (
    df.sort("epoch_time")
    .groupby_dynamic(
        index_column="epoch_time",
        every=GROUPBY_EVERY,
        by=["object_id"],
    )
    .agg(
        [
            pl.col("f32_positionX_m").mean(),
            pl.col("f32_positionY_m").mean(),
            pl.col("f32_velocityInDir_mps").mean(),
            # take the first value of the rest of the columns
            *(
                pl.col(col).first()
                for col in df.columns
                if col
                not in [
                    "f32_positionX_m",
                    "f32_positionY_m",
                    "f32_velocityInDir_mps",
                    "object_id",
                    "epoch_time",
                ]
            ),
        ]
    )
)


In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"
fig = go.Figure()

fig.update_layout(template="ggplot2", height=600, width=1200, font_size=18)


# create a color map for all the radar ips
radar_ips = df["ip"].unique().to_list()
radar_colors = px.colors.qualitative.D3
radar_color_map = {
    ip: radar_colors[i % len(radar_colors)] for i, ip in enumerate(radar_ips)
}

pandas_df = df.to_pandas()

for ip, r in pandas_df.groupby("ip"):
    _s = r.sample(10000)

    fig.add_trace(
        go.Scattermapbox(
            lon=_s["lon"],
            lat=_s["lat"],
            mode="markers",
            opacity=1,
            line_width=4,
            name=ip,
            marker=dict(
                size=10,
                color=radar_color_map[ip],
                opacity=1,
            ),
            visible=True,
        )
    )

fig.update_layout(
    margin=go.layout.Margin(
        l=50,  # left margin
        r=50,  # right margin
        b=50,  # bottom margin
        t=50,  # top margin
    ),
    mapbox=dict(
        accesstoken=mapbox_key,
        bearing=0,
        # style as mapbox satellite
        style="satellite-streets",
        center=go.layout.mapbox.Center(lat=_s["lat"].mean(), lon=_s["lon"].mean()),
        pitch=0,
        zoom=14.1,
    ),
)


fig.show()


### Remove Objects that don't move atleast X meters or spend X seconds on the radar


The current settings here are arbitrary. I'm not sure what the best settings are. I'm also not sure if this is the best way to do this. I'm open to suggestions.


In [8]:
MIN_DISTANCE = 100  # meters
MIN_TIME = 5  # seconds


In [9]:
df = df.filter(
    pl.col("object_id").is_in(
        df.groupby("object_id")
        .agg(
            [
                # calculate the distance between the first and last position
                (
                    (
                        pl.col("f32_positionX_m").first()
                        - pl.col("f32_positionX_m").last()
                    ).pow(2)
                    + (
                        pl.col("f32_positionY_m").first()
                        - pl.col("f32_positionY_m").last()
                    ).pow(2)
                )
                .sqrt()
                .alias("straight_distance"),
                (
                    (pl.col("f32_positionX_m").diff()) ** 2
                    + (pl.col("f32_positionY_m").diff()) ** 2
                )
                .sqrt()
                .sum()
                .alias("distance"),
                # calculate the time between the first and last position
                (pl.col("epoch_time").last() - pl.col("epoch_time").first())
                .dt.seconds()
                .alias("duration"),
            ]
        )
        .filter(
            (pl.col("straight_distance") >= MIN_DISTANCE)
            & (pl.col("duration") >= MIN_TIME)
        )["object_id"]
        .to_list()
    )
)


#### Filter Out The Artifact at End of Trajectory Where Velocity == Itself


In [10]:
df = (
    df.sort("epoch_time")
    .with_columns(
        [
            (
                (pl.col("f32_velocityInDir_mps").diff().abs() < 0.01).fill_null(True)
                & (pl.col("f32_velocityInDir_mps") > 0)
            )
            .over("object_id")
            .alias("stopped"),
        ]
    )
    .with_columns(
        [
            (~pl.col("stopped"))
            .cast(pl.Int8())
            .cumsum()
            .over("object_id")
            .alias("stopped_count")
        ]
    )
    .with_columns(
        (pl.col("stopped_count") >= pl.col("stopped_count").max())
        .over("object_id")
        .alias("trim")
    )
    .filter(~pl.col("trim"))
    .sort(["object_id", "epoch_time"])
)


### Geolocate the Radar Data


In [11]:
# create the file paths
network_outline_file = "./geo_data/network_outline.geojson"
radar_locations_file = "./geo_data/radar_origins.json"


In [12]:
f = Filtering(
    radar_location_path=radar_locations_file,
    network_boundary_path=network_outline_file,
)


In [13]:
df = (
    df
    # df.clone()
    # .pipe(f.rotate_radars)
    # .pipe(f.radar_to_utm)
    # .pipe(f.radar_to_latlon)
    # .pipe(f.radar_to_h3)
    .pipe(f.int_h3_2_str).pipe(f.filter_network_boundaries)
)


function: int_h3_2_str took: 0.24734115600585938 seconds
function: filter_network_boundaries took: 0.04175400733947754 seconds


## Testing Code

In [14]:
from filtering import Fusion, RadarHandoff

In [15]:
AIRPORT = "10.160.7.136", "10.160.7.137"
LOWES = "10.160.7.141", "10.160.7.142"
HARPER = "10.160.7.146", "10.160.7.147"

In [16]:
fuser = Fusion(
    overlap_threshold=0.001,
    radar_handoffs=[
        RadarHandoff(to=to, from_=from_) 
        for from_, to in [
                (AIRPORT[1], LOWES[0]), 
                (LOWES[1], HARPER[0]),
                # (AIRPORT[0], AIRPORT[1]),
                (LOWES[1], LOWES[0]),
                (LOWES[0], LOWES[1]),
                # (HARPER[0], HARPER[1]),
            ]
    ]
)

fuser.build_overlaps(df)

In [17]:
fuser.get_overlaps().to_pandas().to_csv("overlaps.csv", index=False)

In [25]:
matches = fuser.merge_radars(df, )

# set the match limit to 
# merged_ids = merged_ids.filter(pl.col("distance") < 4)

Joining:  10.160.7.137 -> 10.160.7.141
function: _merge_radar took: 0.333845853805542 seconds
function: _hungarian_match took: 0.0548100471496582 seconds
Joining:  10.160.7.142 -> 10.160.7.146
function: _merge_radar took: 0.22259998321533203 seconds
function: _hungarian_match took: 0.010857105255126953 seconds
Joining:  10.160.7.142 -> 10.160.7.141
function: _merge_radar took: 0.05048394203186035 seconds
function: _hungarian_match took: 0.005937099456787109 seconds
Joining:  10.160.7.141 -> 10.160.7.142
function: _merge_radar took: 0.025288105010986328 seconds
function: _hungarian_match took: 0.008580207824707031 seconds
function: merge_radars took: 0.7250511646270752 seconds


In [ ]:
# target_pair = RadarHandoff(to=LOWES[1], from_=LOWES[0])

# overlap_df = df.filter(pl.col("h3").is_in(fuser._overlap_h3s[target_pair]))

# tmp = (
#     overlap_df.filter(pl.col("ip") == target_pair.to)
#     .join(
#         overlap_df.filter(pl.col("ip") == target_pair.from_).select(
#             [
#                 "h3",
#                 "epoch_time",
#                 "object_id",
#                 "utm_x",
#                 "utm_y",
#                 "f32_velocityInDir_mps",
#                 "f32_length_m",
#             ]
#         ),
#         on="epoch_time",
#         how="inner",
#         suffix="_search",
#     )
#     .with_columns(
#         [
#             (pl.col("utm_x") - pl.col("utm_x_search")).alias("x_diff"),
#             (pl.col("utm_y") - pl.col("utm_y_search")).alias("y_diff"),
#             (
#                 pl.col("f32_velocityInDir_mps")
#                 - pl.col("f32_velocityInDir_mps_search")
#             ).alias("v_diff"),
#         ]
#     )
#     # standard scale the data
#     .with_columns(
#         [
#             (
#                 pl.col(["x_diff", "y_diff", "v_diff"])
#                 - pl.col(["x_diff", "y_diff", "v_diff"]).min()
#             )
#             / (pl.col(["x_diff", "y_diff", "v_diff"]).max() - pl.col(["x_diff", "y_diff", "v_diff"]).min()),
#         ]
#     )
#     .with_columns(
#         [
#             (
#                 pl.col("x_diff").pow(2)
#                 + pl.col("y_diff").pow(2)
#                 + pl.col("v_diff").pow(2)
#             )
#             .sqrt()
#             .alias("distance"),
#         ]
#     )
#     # .groupby(["object_id_search", "object_id"])
#     # .agg(
#     #     [
#     #         # calculate the euclidean distance between the a vection of x, y, and velocity
#     #         pl.col("distance")
#     #         # use the median distance to account for outliers
#     #         .min()
#     #         .alias("distance"),
#     #     ]
#     # )
#     .select(["object_id_search", "object_id", "distance", "epoch_time", "x_diff", "y_diff", "v_diff"])
# )

In [26]:

matches.to_pandas().query("distance<0.1")['distance'].hist(bins=100)

In [30]:
matches.sort("distance", descending=True).filter(pl.col("distance") <0.1).to_pandas().head(10)

,object_id_search,object_id,distance
0,45218_10.160.7.142,74624_10.160.7.146,0.098617
1,45437_10.160.7.142,74987_10.160.7.146,0.096185
2,46352_10.160.7.137,27840_10.160.7.141,0.093642
3,45342_10.160.7.142,27626_10.160.7.141,0.091241
4,27626_10.160.7.141,45342_10.160.7.142,0.091241
5,45192_10.160.7.142,27408_10.160.7.141,0.090317
6,27408_10.160.7.141,45192_10.160.7.142,0.090317
7,45079_10.160.7.142,27341_10.160.7.141,0.088218
8,27341_10.160.7.141,45079_10.160.7.142,0.088218
9,45097_10.160.7.142,27340_10.160.7.141,0.087598


In [33]:
# plot the velocity of the objects
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# objs = 	"40595_10.160.7.136", "46405_10.160.7.137"
objs = ("46352_10.160.7.137",	"27840_10.160.7.141")

# failed_match = "46234_10.160.7.137", "61830_10.160.7.141", "61833_10.160.7.141"


# create two subplots, one for velocity and one for position, with plotly
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Velocity", "Position"),
)

for obj in objs:
    plot_df = df.filter(pl.col("object_id") == obj).to_pandas()

    # plot the velocity
    fig.add_trace(
        go.Scatter(
            x=plot_df["epoch_time"],
            y=plot_df["f32_velocityInDir_mps"],
            name=obj,
        ),
        row=1,
        col=1,
    )

    # plot the position
    fig.add_trace(
        go.Scatter(
            x=plot_df["utm_x"],
            y=plot_df["utm_y"],
            name=obj,
        ),
        row=1,
        col=2,
    )

    # set the colors of the traces to the same color
    fig.data[-2].line.color = fig.data[-1].line.color


fig.show()


In [34]:
import folium
from folium import plugins


m = folium.Map(
    location=[df["lat"].mean(), df["lon"].mean()],
    zoom_start=16,
    max_zoom=20,
    height=500,
    width=1200,
)

color_map = {o: c for o, c in zip(objs, ["red", "blue", "green", "orange", "yellow"])}


# get the min and max time
min_time = df.filter(pl.col("object_id").is_in(objs))["epoch_time"].min()
max_time = df.filter(pl.col("object_id").is_in(objs))["epoch_time"].max()

lines = (
    df.filter(pl.col("epoch_time").is_between(min_time, max_time))
    # .filter(pl.col("ip").is_in(LOWES + AIRPORT))
    .filter(pl.col("object_id").is_in(objs))
    .to_pandas()
    .groupby(
        [
            "object_id",
        ]
    )
    .apply(
        lambda x: {
            "coordinates": x[["lon", "lat"]].values.tolist(),
            "dates": x["epoch_time"].apply(lambda x: f"{x}").tolist(),
            "color": color_map.get(x["object_id"].values[0], "purple"),
            "tooltip": f"{x['object_id'].values[0]}",
            "weight": 5 if x["object_id"].values[0] in objs else 1,
        }
    )
    .to_list()
)

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"]
            },
            "tooltip": line["tooltip"] if "tooltip" in line else None,
        },
    }
    for line in lines
]


plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1S",
    add_last_point=True,
    time_slider_drag_update=True,
    # transition_time=1
).add_to(m)


# set the size of the map to be 1000x1000 pixels
m


In [91]:
import pytz

pd.to_datetime(min_time, ).tz_localize(pytz.utc).tz_convert("US/Central")

Timestamp('2023-03-12 17:06:27.100000-0500', tz='US/Central')

### Automatically Find the Regions of Overlapping Data


In [52]:
# overlaps = (
#     df.groupby(["h3", "ip"])
#     .agg(
#         [
#             pl.col("object_id").count().alias("count"),
#         ]
#     )
#     .pivot(values="count", index="h3", columns="ip", aggregate_function="sum")
#     .fill_null(0)
#     .to_pandas()
#     .set_index("h3")
# )

# # divide each row by its sum
# overlaps = overlaps.div(overlaps.sum(axis=1), axis=0)


### Testing Stuff


#### Creating Overlap Zones


In [53]:
# AIRPORT = "10.160.7.136", "10.160.7.137"
# LOWES = "10.160.7.141", "10.160.7.142"
# HARPER = "10.160.7.146", "10.160.7.147"

# # overlapping_pairs = ((AIRPORT[1], LOWES[0]),)
# overlapping_pairs = ((LOWES[1], HARPER[0]),)



In [54]:
# # create a set of h3 corresponding to the overlapping pairs
# THRESHOLD = 0.1

# overlapping_h3 = {
#     pair: list(
#         set(overlaps.loc[overlaps[pair[0]] > THRESHOLD].index)
#         & set(overlaps.loc[overlaps[pair[1]] > THRESHOLD].index)
#     )
#     for pair in overlapping_pairs
# }


#### Testing on a Single Zone

- Open improvement points
  - Use the length of the vehicle to better determine whether the vehicle is overlapping
  - Use a probability based approach
    - We assume that a vehicle in the overlap zone will have an overlap 


In [55]:
# # df.filter(pl.col('h3').is_in(overlapping_h3[overlapping_pairs[0]]))

# # get a overlap_df
# overlap_df = df.filter(pl.col("h3").is_in(overlapping_h3[overlapping_pairs[0]]))


In [56]:
# # create separate dataframes for each radar
# radar_dfs = {
#     radar: overlap_df.filter(pl.col("ip") == radar) for radar in overlapping_pairs[0]
# }


In [57]:
# import h3


# r1 = overlapping_pairs[0][0]
# r2 = overlapping_pairs[0][1]


# distances = (
#     radar_dfs[r2]
#     .join(
#     radar_dfs[r1].select(
#             ["h3", "epoch_time", "object_id", "utm_x", "utm_y", "f32_velocityInDir_mps", "f32_length_m"]
#         ),
#         on="epoch_time",
#         how="inner",
#         suffix="_search",
#     )
#     .with_columns(
#         [
#             (pl.col("utm_x") - pl.col("utm_x_search")).alias("x_diff"),
#             (pl.col("utm_y") - pl.col("utm_y_search")).alias("y_diff"),
#             (pl.col("f32_velocityInDir_mps") - pl.col("f32_velocityInDir_mps_search")).alias("v_diff"),
#         ]
#     )
#     # standard scale the data
#     .with_columns(
#         [
#             (pl.col(["x_diff", "y_diff", "v_diff"]) - pl.col(["x_diff", "y_diff", "v_diff"]).mean()) / pl.col(["x_diff", "y_diff", "v_diff"]).std(),
#         ]
#     )
#     .with_columns(
#         [
#             (pl.col("x_diff").pow(2) + pl.col("y_diff").pow(2) + pl.col("v_diff").pow(2)).sqrt().alias("distance"),
#         ]
#     )
#     .groupby(["object_id_search", "object_id"])
#     .agg(
#         [
#             # pl.col("velocity_diff").mean().alias("velocity_diff"),
#             # pl.col("position_diff").mean().alias("position_diff"),
#             # # also weight by the number of points in each object
#             # pl.col("velocity_diff").count().alias("weight"),
#             # # overlaps == when the positional difference is less than legnth of the object
#             # # (1 - ((pl.col("position_diff") < pl.col("f32_length_m_search")).sum() / pl.col("velocity_diff").count())).alias("overlap"),
#             # ((pl.col("position_diff") < pl.col("f32_length_m_search")).sum() / pl.col("object_id").count()).alias("overlap"),
        
#             # calculate the euclidean distance between the a vection of x, y, and velocity
#             pl.col("distance").mean().alias("distance"),
#         ]
#     )
#     # have to group it again by the object id because several objects may match the same search object and we want to pick the best match
#     .sort("distance")
#     # .groupby(
#     #     ["object_id"]
#     # )
#     # .head(1)
# )

# cost_matrix = distances.pivot(index="object_id_search", columns="object_id", values="distance").fill_null(1e9)

# cost_matrix = cost_matrix.select([c for c in cost_matrix.columns if c != "object_id_search"]).to_numpy().astype(np.float64)


# import scipy.optimize as opt

# r, c = opt.linear_sum_assignment(cost_matrix)

# # create a dataframe with the matches
# matches = pl.DataFrame(
#     {
#         "object_id_search": distances["object_id_search"].take(r),
#         "object_id": distances["object_id"].take(c),
#         "distance": distances["distance"].take(c),
#     }
# )





In [32]:
# fig = go.Figure()

# distances_np = distances["distance"].to_numpy()

# fig.add_trace(
#     go.Histogram(
#         x=distances_np,
#         histnorm="probability",
#         name="distance",
#         # set the opacity to 0.5
#         opacity=0.5,
#     )
# )

# # add the matches
# fig.add_trace(
#     go.Histogram(
#         x=matches["distance"].to_numpy(),
#         histnorm="probability",
#         name="matches",
#         # set the opacity to 0.5
#         opacity=0.5,
#     )
# )

# fig.update_layout(
#     title="Distance between objects",
#     xaxis_title="Distance",
#     yaxis_title="Probability",
#     # stack bars
#     barmode="overlay",
# )



# fig.show()

In [35]:
# plot the velocity of the objects
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# obj1 = "61822_10.160.7.141"
# obj2 = "46230_10.160.7.137"

# objs = ["46371_10.160.7.137", "61959_10.160.7.141", "61958_10.160.7.141"]
# objs = ("46155_10.160.7.137", "46151_10.160.7.137", "27506_10.160.7.141")  # "62303_10.160.7.141"
# objs = ("45101_10.160.7.142",	"74515_10.160.7.146", "45104_10.160.7.142", "45114_10.160.7.142", "74509_10.160.7.146")
# trailer_example = ("45636_10.160.7.142",	"45638_10.160.7.142")
objs = 	"45297_10.160.7.142",	"74772_10.160.7.146"

failed_match = "46234_10.160.7.137", "61830_10.160.7.141", "61833_10.160.7.141"


# create two subplots, one for velocity and one for position, with plotly
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Velocity", "Position"),
)

for obj in objs:
    plot_df = df.filter(pl.col("object_id") == obj).to_pandas()

    # plot the velocity
    fig.add_trace(
        go.Scatter(
            x=plot_df["epoch_time"],
            y=plot_df["f32_velocityInDir_mps"],
            name=obj,
        ),
        row=1,
        col=1,
    )

    # plot the position
    fig.add_trace(
        go.Scatter(
            x=plot_df["utm_x"],
            y=plot_df["utm_y"],
            name=obj,
        ),
        row=1,
        col=2,
    )

    # set the colors of the traces to the same color
    fig.data[-2].line.color = fig.data[-1].line.color


fig.show()


In [38]:
import folium
from folium import plugins


m = folium.Map(
    location=[df["lat"].mean(), df["lon"].mean()],
    zoom_start=16,
    max_zoom=20,
    height=500,
    width=1200,
)

color_map = {o: c for o, c in zip(objs, ["red", "blue", "green"])}


# get the min and max time
min_time = df.filter(pl.col("object_id").is_in(objs))["epoch_time"].min()
max_time = df.filter(pl.col("object_id").is_in(objs))["epoch_time"].max()

lines = (
    # df.filter(pl.col("epoch_time").is_between(min_time, max_time))
    df.filter(pl.col("object_id").is_in(objs))
    .to_pandas()
    .groupby(
        [
            "object_id",
        ]
    )
    .apply(
        lambda x: {
            "coordinates": x[["lon", "lat"]].values.tolist(),
            "dates": x["epoch_time"].apply(lambda x: f"{x}").tolist(),
            "color": color_map.get(x["object_id"].values[0], "purple"),
            "tooltip": f"{x['object_id'].values[0]}",
        }
    )
    .to_list()
)

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 5,
            },
            "tooltip": line["tooltip"] if "tooltip" in line else None,
        },
    }
    for line in lines
]


plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1S",
    add_last_point=True,
    time_slider_drag_update=True,
    # transition_time=1
).add_to(m)


# set the size of the map to be 1000x1000 pixels
m


In [ ]:
# x = s.groupby("object_id_search", "object_id").agg(
#     [
#         pl.col("h3_distance").mean().alias("mean"),
#     ]
# ).sort(
#     ["object_id_search", "mean"],
# ).groupby(
#     "object_id_search", maintain_order=True
# ).agg(
#     [
#         pl.col("object_id").first(),
#         pl.col("mean").first(),
#     ]
# ).to_pandas()['mean']


# fig = go.Figure()

# fig.add_trace(
#     go.Histogram(
#         x=x,
#         name="Distance",
#         nbinsx=100,
#     )
# )

# fig.show()


In [ ]:
x = (
    s.groupby("object_id_search", "object_id")
    .agg(
        [
            pl.col("h3_distance").mean().alias("mean"),
            (
                (pl.col("x") - pl.col("x_search")).pow(2)
                + (pl.col("y") - pl.col("y_search")).pow(2)
            )
            .mean()
            .sqrt()
            .alias("distance"),
        ]
    )
    .sort(
        ["object_id_search", "mean"],
    )
    .groupby("object_id_search", maintain_order=True)
    .agg(
        [
            # pl.when(pl.col('object_id').n_unique() > 1).then(pl.col("object_id").take(1)).otherwise(pl.lit(None)).alias("object_id"),
            # pl.when(pl.col('object_id').n_unique() > 1).then(pl.col("mean").take(1)).otherwise(pl.lit(None)).alias("mean"),
            pl.col("object_id").head(5),
            pl.col("mean").head(5),
            pl.col("distance").head(5),
        ]
    )
    .to_pandas()
)


In [ ]:
# x = x.sort_values("mean", ascending=True)
# x.loc[x["mean_1"] > 2,]
x.head()


In [ ]:
x.loc[x["object_id_search"] == "46677_10.160.7.137",]


In [ ]:
x.loc[x["object_id"] == "62303_10.160.7.141"]
